
<div class="box">
  <table>
      <tr>
          <th colspan='2'><h1 style="text-align: center">
Foundations of DataScience </br>
</h1>
<h2 style="text-align: center">
Course Project Bonus Scraper
    </br>
</h2>
    
    

</th>
<tr>
    <tr>
      <th colspan="2">Personal Info</th>
    </tr>
    <tr>
      <td>First Name:</td>
      <td>Ali</td>
    </tr>
    <tr>
      <td>Last Name:</td>
      <td>Nikkhah</td>
    </tr>
    <tr>
      <td>Student Number:</td>
      <td>99102445</td>
    </tr>
    <tr>
      <td>Git:</td>
      <td><a href="https://github.com/AliNikkhah2001/DataScience02" target="_blank">https://github.com/AliNikkhah2001/DataScience02</a></td>
    </tr>
  </table>
</div>


In [1]:
import requests

def check_connection(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Check for HTTP errors
        print(f"Connection to {url} successful. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error during connection to {url}: {e}")

# List of sample websites
websites = [
    "https://www.digikala.com/search/notebook-netbook-ultrabook/",
    "https://www.zillow.com/homes/for_rent/apartment_duplex_type/",
    "https://divar.ir/s/tehran",
    "https://zapier.com/apps",
]

# Check connection to each website
for website in websites:
    check_connection(website)


Connection to https://www.digikala.com/search/notebook-netbook-ultrabook/ successful. Status code: 200
Error during connection to https://www.zillow.com/homes/for_rent/apartment_duplex_type/: 403 Client Error: Forbidden for url: https://www.zillow.com/homes/for_rent/apartment_duplex_type/
Connection to https://divar.ir/s/tehran successful. Status code: 200
Connection to https://zapier.com/apps successful. Status code: 200


<h1> Automatic click setup</h1>


In [4]:
!pip install selenium
!pip install chromedriver_autoinstaller



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
import time


In [3]:
import urllib3
import ssl
from IPython.core.display import display, HTML

# Ignore SSL certificate warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
ssl._create_default_https_context = ssl._create_unverified_context

# Display a message to inform users that SSL errors are being ignored
display(HTML("<div style='color: red; font-weight: bold;'>SSL certificate errors are being ignored.</div>"))


/tmp/ipykernel_56908/1697730047.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.chrome.options import Options

# Automatically install and set up the latest version of ChromeDriver
chromedriver_autoinstaller.install()

# Set up the WebDriver with Chrome options to ignore SSL errors
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Set up the WebDriver with the defined options
driver = webdriver.Chrome(options=chrome_options)


In [5]:
!mkdir -p /Users/alinikkhah/.cache/selenium
!chmod -R 777 /Users/alinikkhah/.cache/selenium


In [37]:

def click_load_more(driver):
    try:
        # Find the "Load more" button by class name
        load_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "css-i5wwsv-CategoryAppTable__loadMore"))
        )
        # Scroll into view to make sure the button is clickable
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        
        # Click the "Load more" button
        load_more_button.click()
        
        # Wait for a moment to let new items load
        time.sleep(2)
        
        return True
    except Exception as e:
        print(f"Error clicking 'Load more' button: {e}")
        return False

# Automatically install and set up the latest version of ChromeDriver
chromedriver_autoinstaller.install()

# Set up the WebDriver with Chrome options
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the Zapier apps page
driver.get('https://zapier.com/apps')

for _ in range(5):  # Click 5 times as an example
    if click_load_more(driver):
        print("Successfully clicked 'Load more' button.")
    else:
        print("Failed to click 'Load more' button. Exiting.")
        break

# Close the WebDriver
driver.quit()


Successfully clicked 'Load more' button.
Successfully clicked 'Load more' button.
Successfully clicked 'Load more' button.
Successfully clicked 'Load more' button.
Successfully clicked 'Load more' button.


<h1> Scraper</h1>
<h2>Plain simple extraction of data 5%</h2>

In [21]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm.notebook import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import os
from urllib.parse import urlparse

def extract_page_data(url, session, output_folder):
    try:
        response = session.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        software_list = []

        # Use data-testid attribute to find the main div
        grid_elements = soup.find('div', {'data-testid': 'category-top-apps-list'}).select('li.css-1qwpe4d')

        for element in grid_elements:
            software_data = {}

            # Extract data from each element within the grid
            title_elem = element.find('h3', class_='css-a7cy59-Text--paragraph3Bold--inherit')
            
            # Find the span with data-testid="ServiceIconShell"
            img_span = element.find('span', {'data-testid': 'ServiceIconShell'})
            
            # If the span is found, find the img tag and extract its src attribute
            if img_span:
                img_tag = img_span.find('img')
                img_src = img_tag['src'] if img_tag and 'src' in img_tag.attrs else None
            else:
                img_src = None

            summary_elem = element.find('p', class_='css-1qmcnjr-CategoryAppCard__description')

            if title_elem:
                software_data['Title'] = title_elem.text.strip()
            else:
                software_data['Title'] = 'N/A'

            software_data['Image'] = img_src if img_src else 'N/A'

            if summary_elem:
                software_data['Summary'] = summary_elem.text.strip()
            else:
                software_data['Summary'] = 'N/A'

            software_list.append(software_data)

        return software_list
    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return []


def download_image(url, output_folder, title):
    try:
        response = requests.get(url)
        response.raise_for_status()

        # Create output folder if it doesn't exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Extract filename from URL
        filename = os.path.join(output_folder, f"{title}.png")
        
        # Save image to file
        with open(filename, 'wb') as f:
            f.write(response.content)

        print(f"Image '{title}.png' downloaded successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")


def save_to_csv(data, filename):
    keys = data[0].keys()

    with open(filename, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=keys)
        writer.writeheader()
        writer.writerows(data)

def crawl_website(output_folder):

    all_software_data = []

    session = requests.Session()
    retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    session.mount('http://', HTTPAdapter(max_retries=retries))
    session.mount('https://', HTTPAdapter(max_retries=retries))
    url = 'https://zapier.com/apps/'
    page_data = extract_page_data(url, session,output_folder)
    all_software_data.extend(page_data)
    return all_software_data



In [22]:

# Crawl the website and save data to CSV
output_folder = 'downloaded_images'
output_filename = 'zapier_software_data.csv'

all_data = crawl_website(output_folder)

if all_data:
    print("Data successfully retrieved.")
    save_to_csv(all_data, output_filename)
else:
    print("No data retrieved. Please check your connection or the website structure.")
if all_data:
    print("Data successfully retrieved.")
    save_to_csv(all_data, output_filename)
else:
    print("No data retrieved. Please check your connection or the website structure.")


Data successfully retrieved.
Data successfully retrieved.


In [23]:
print(all_data)

[{'Title': 'Google Sheets', 'Image': 'https://zapier-images.imgix.net/storage/services/8913a06feb7556d01285c052e4ad59d0.png?auto=format&ixlib=react-9.8.0&q=50&fit=crop&h=20&w=20', 'Summary': 'Create, edit, and share spreadsheets wherever you are with Google Sheets, and get automated insights from your data.'}, {'Title': 'Gmail', 'Image': 'https://zapier-images.imgix.net/storage/services/1afcb319c029ec5da10efb593b7159c8.png?auto=format&ixlib=react-9.8.0&q=50&fit=crop&h=20&w=20', 'Summary': 'One of the most popular email services, Gmail keeps track of all your emails with threaded conversations, tags, and Google-powered search to find any message you need.'}, {'Title': 'Filter by Zapier', 'Image': 'https://zapier-images.imgix.net/storage/services/ad3d7962908c17bcbe753928e8786b4a.png?auto=format&ixlib=react-9.8.0&q=50&fit=crop&h=20&w=20', 'Summary': "Only allow a Zap to proceed when a certain condition is met. For example, if you're sending a text message when you receive a new email, you

<h1> Combining both to get more data and open each data seprate page</h1>
<h2>If you script performs scrolling the page to load more content or clicks on buttons to load the next page 5%.
</h2>

In [54]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import matplotlib.pyplot as plt
import uuid
import csv

def click_load_more(driver):
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "css-i5wwsv-CategoryAppTable__loadMore"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        time.sleep(5)  # Wait for 5 seconds after each click
        return True
    except Exception as e:
        print(f"Error clicking 'Load more' button: {e}")
        return False

def scrape_data(driver, output_folder):
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        software_list = []

        grid_elements = soup.find_all('li', class_='css-1qwpe4d')


        for element in grid_elements:
            software_data = {}

            title_elem = element.find('h3', class_='css-a7cy59-Text--paragraph3Bold--inherit')
            img_span = element.find('span', {'data-testid': 'ServiceIconShell'})
            
            if img_span:
                img_tag = img_span.find('img')
                img_src = img_tag['src'] if img_tag and 'src' in img_tag.attrs else None
            else:
                img_src = None

            summary_elem = element.find('p', class_='css-1qmcnjr-CategoryAppCard__description')

            if title_elem and img_src and summary_elem:
                software_data['Title'] = title_elem.text.strip()
                software_data['Image'] = img_src
                software_data['Summary'] = summary_elem.text.strip()

                # Generate a unique ID for each entry
                software_data['ID'] = str(uuid.uuid4())

                software_list.append(software_data)

                # Download the image
                download_image(img_src, output_folder, software_data['ID'])

        return software_list
    except Exception as e:
        print(f"Error during scraping: {e}")
        return []

def download_image(url, output_folder, identifier):
    try:
        if url:
            response = requests.get(url)
            response.raise_for_status()

            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            filename = os.path.join(output_folder, f"{identifier}.png")

            with open(filename, 'wb') as f:
                f.write(response.content)

            print(f"Image '{identifier}.png' downloaded successfully.")
        else:
            print(f"No image URL provided for '{identifier}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
def display_samples(samples):
    num_samples = len(samples)
    
    fig, axs = plt.subplots(num_samples, 1, figsize=(10, 3 * num_samples))
    
    for i, sample in enumerate(samples):
        img_filename = f"{sample['ID']}.png"
        img_path = os.path.join('downloaded_images', img_filename)
        
        try:
            img = plt.imread(img_path)
            axs[i].imshow(img)
            axs[i].axis('off')
            
            title = f"Title: {sample['Title']}\n"
            description = f"Description: {sample['Summary']}"
            axs[i].set_title(title + description)
        except Exception as e:
            print(f"Error loading image {img_filename}: {e}")
            axs[i].axis('off')  # Hide the axis if there's an error
        
    plt.tight_layout()
    plt.show()

def save_and_display_samples(scraped_data, output_folder):
    # Save the scraped data to a CSV file
    csv_filename = os.path.join(output_folder, 'scraped_data.csv')
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ID', 'Title', 'Image', 'Summary']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(scraped_data)
    
    print(f"Scraped data saved to {csv_filename}.")

    # Display samples using the provided function
    display_samples(scraped_data)

# Your existing code for setting up ChromeDriver and navigating to the page
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-ssl-errors=yes')
chrome_options.add_argument('--ignore-certificate-errors')

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://zapier.com/apps')

output_folder = 'downloaded_images'

scraped_data = []

# Click the "Load more" button 20 times with a 5-second delay between each click
for _ in range(5):
    if click_load_more(driver):
        print(f"Successfully clicked 'Load more' button {_ + 1} times.")
        
    else:
        print("Failed to click 'Load more' button. Exiting.")
        break
# Scrape data from the newly loaded content and append to the existing data
scraped_data = scrape_data(driver, output_folder)
# Save and display the scraped data
save_and_display_samples(scraped_data, output_folder)

driver.quit()


Successfully clicked 'Load more' button 1 times.
Successfully clicked 'Load more' button 2 times.
Successfully clicked 'Load more' button 3 times.
Successfully clicked 'Load more' button 4 times.
Successfully clicked 'Load more' button 5 times.
Image '6e591e84-9edf-431a-a48c-d4f1b56be9fa.png' downloaded successfully.
Image '3b432790-2dbd-41bf-bf6a-f32252b97446.png' downloaded successfully.
Image '4dc7fdf0-8401-456a-93c6-a4db0829de84.png' downloaded successfully.
Image '1c3a9772-8475-4672-a5a5-4d3b4bb7e919.png' downloaded successfully.
Image '2864343e-2d62-42c4-a550-ef6bd0a432ed.png' downloaded successfully.
Image 'd9b13c46-cc54-46ae-9221-f063417beeb2.png' downloaded successfully.
Image '5c6bb738-7ab3-45e3-b028-8542acb60413.png' downloaded successfully.
Image '7767345f-2633-4d96-86b1-645509528d46.png' downloaded successfully.
Image '6f4c9d0a-a606-4a42-8692-fd0c177d89d9.png' downloaded successfully.
Image '7c43533b-7d96-4f14-a469-f6358dba7ab8.png' downloaded successfully.
Image 'ebe9212c

) missing from current font.876.py:107: UserWarning: Glyph 13 (
  plt.tight_layout()
